In [0]:
%pylab inline

In [2]:
import dataiku
import dataiku.spark as dkuspark
import pyspark
from pyspark.sql import SQLContext

/home/dataiku/dataiku-dss-8.0.1/spark-standalone-home/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py:2020: DeprecationWarning: invalid escape sequence \*
/home/dataiku/dataiku-dss-8.0.1/spark-standalone-home/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py:2020: DeprecationWarning: invalid escape sequence \*


In [3]:
# Load PySpark
sc = pyspark.SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [4]:
## Spark NLP imports
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import Tokenizer, Normalizer, LemmatizerModel, StopWordsCleaner
from pyspark.ml import Pipeline
import sparknlp

## NLTK import
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
eng_stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dataiku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
combined_raw = dataiku.Dataset("combined_raw")
combined_raw_df = dkuspark.get_dataframe(sqlContext, combined_raw)

document = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')
    
token = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
    
normal = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)
    
lemmatizer = LemmatizerModel.load("/home/dataiku/lemma/") \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma')
    
stopwords = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords)
    
finish = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)
    
pipeline = Pipeline().setStages([
    document,
    token,
    normal,
    lemmatizer,
    stopwords,
    finish
])


combined_cleaned_df = pipeline.fit(combined_raw_df).transform(combined_raw_df)

In [6]:
combined_cleaned_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nu

In [7]:
combined_cleaned = dataiku.Dataset("combined_cleaned")
combined_cleaned_df = dkuspark.get_dataframe(sqlContext, combined_cleaned)

In [10]:
from pyspark.sql.functions import explode, col

words = combined_cleaned_df.withColumn('exploded_text', explode(col('finished_clean_lemma'))) \
    .groupby('title', 'exploded_text').count()

In [11]:
words.show(n=5)

+--------------------+-------------+-----+
|               title|exploded_text|count|
+--------------------+-------------+-----+
|The Pilgrims of N...|       follow|  127|
|The Pilgrims of N...|   toleration|    4|
|The Pilgrims of N...|      abandon|   24|
|The Pilgrims of N...|        piety|   17|
|The Pilgrims of N...|         wave|   23|
+--------------------+-------------+-----+
only showing top 5 rows

